In [2]:
    open Microsoft.Quantum.Canon;
    open Microsoft.Quantum.Intrinsic;
    open Microsoft.Quantum.ErrorCorrection;
    open Microsoft.Quantum.Math;
    open Microsoft.Quantum.Diagnostics;
    open Microsoft.Quantum.Arithmetic;
    open Microsoft.Quantum.Synthesis;
    
    operation prepare_secret(q : Qubit[]) : Unit {
        let theta = 1.0471975;// = pi / 3
        let matrix = [[Complex(Cos(theta), 0.0), Complex(0.0, Sin(theta))],
                      [Complex(0.0, Sin(theta)), Complex(Cos(theta), 0.0)]];
        let register = LittleEndian(q);
        ApplyUnitary(matrix, register);
    }

prepare_secret

In [3]:
    operation erasure_correcting_procedure(fifth : Qubit,fourth : Qubit,third : Qubit) : Qubit[]{
        let qecc = FiveQubitCode();
        let recoveryfn = TableLookupRecovery([
            [PauliI,PauliI,PauliI,PauliI,PauliI],
            [PauliI,PauliI,PauliI,PauliY,PauliY],
            [PauliI,PauliI,PauliI,PauliI,PauliZ],
            [PauliI,PauliI,PauliI,PauliY,PauliX],

            [PauliI,PauliI,PauliI,PauliX,PauliZ],
            [PauliI,PauliI,PauliI,PauliZ,PauliX],
            [PauliI,PauliI,PauliI,PauliX,PauliI],
            [PauliI,PauliI,PauliI,PauliZ,PauliY],

            [PauliI,PauliI,PauliI,PauliX,PauliY],
            [PauliI,PauliI,PauliI,PauliZ,PauliI],
            [PauliI,PauliI,PauliI,PauliX,PauliX],
            [PauliI,PauliI,PauliI,PauliZ,PauliZ],

            [PauliI,PauliI,PauliI,PauliI,PauliX],
            [PauliI,PauliI,PauliI,PauliY,PauliZ],
            [PauliI,PauliI,PauliI,PauliI,PauliY],
            [PauliI,PauliI,PauliI,PauliY,PauliI]
        ]);

        use a = Qubit();
        use b= Qubit();

        let Logical0 = LogicalRegister([fifth,fourth,third,b,a]);

        Recover(qecc,recoveryfn,Logical0);

        let (secret,scratch0) = DecodeFromFiveQubitCode(Logical0);
        return secret;
    }

erasure_correcting_procedure

In [4]:
    operation main_erasure() : Unit {

        use data = Qubit[1];//prepare 1qubit
        prepare_secret(data);//prepare secret
        DumpMachine();//display secret

        use scratch = Qubit[4];//prepare scratch

        let Logical = EncodeIntoFiveQubitCode(data,scratch);//encode

        let secret=erasure_correcting_procedure(data[0],scratch[0],scratch[1]);//reconstruct secret
        Message("---------------");
        DumpRegister((),secret);//display reconstructed secret
    }

main_erasure

In [5]:
    %trace main_erasure //simulate secret reconstructions

Released qubits are not in zero state.


In [6]:
//----The following is for visualising the circuit-----------------------
    operation visualising_erasure_circuit() : Unit{
        use fifth = Qubit();
        use fourth = Qubit();
        use third = Qubit();
        
        let secret=erasure_correcting_procedure(fifth,fourth,third);
        
        Ignore(M(fifth));
        Ignore(M(fourth));
        Ignore(M(third));
    }


visualising_erasure_circuit

In [7]:
%trace visualising_erasure_circuit //visualising the circuit

DisplayableHtmlElement { Html = <div id='execution-path-container-a91809d9-33ac-421f-b54f-fa7124fad081' /> }